In [12]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix,f1_score,classification_report
import numpy as np
import re,unicodedata,nltk
from nltk.corpus import stopwords
from transformers import AutoTokenizer, TFXLMRobertaForSequenceClassification 
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import keras
# import transformers
# import tensorflow_hub as hub
# from tqdm import tqdm
# import pickle
# from keras.models import Model
# import keras.backend as K
# from sklearn.metrics import confusion_matrix,f1_score,classification_report
# import matplotlib.pyplot as plt
# from keras.callbacks import ModelCheckpoint
# import itertools
# from keras.models import load_model
# from transformers import TrainingArguments, Trainer

nltk.download('stopwords')
stops_nltk = nltk.corpus.stopwords.words('portuguese')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\roger\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    for gpu in gpus:
        print("Found a GPU with the name:", gpu)
else:
    print("Failed to detect a GPU.")

Found a GPU with the name: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [3]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def clean_stopwords_shortwords(w):
    stopwords_list=stopwords.words('portuguese')
    words = w.split()
    clean_words = [word for word in words if (word not in stopwords_list) and len(word) > 2]
    return " ".join(clean_words)

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r"([?.!,¿])", r" ", w)
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    # w=clean_stopwords_shortwords(w)
    w=re.sub(r'@\w+', '',w)
    return w

In [4]:
df = pd.read_excel("querys.xlsx",sheet_name='Planilha1')
df_to_use = df.loc[:,["Question","NEW INTENT"]].dropna()
data=df_to_use.rename(columns = {'NEW INTENT': 'label', 'Question': 'text'}, inplace = False)

In [5]:
data=data.dropna()                                                           # Drop NaN valuues, if any
data=data.reset_index(drop=True)                                             # Reset index after dropping the columns/rows with NaN values
data = shuffle(data)                                                         # Shuffle the dataset
print('Available labels: ',data.label.unique())                              # Print all the unique labels in the dataset
data['text']=data['text'].map(preprocess_sentence)
data['gt'] = pd.factorize(data['label'], sort=True)[0] + 1

Available labels:  ['QUANTIDADE_DE_DEPARTAMENTOS' 'CARGO_DO_FUNCIONARIO'
 'LOCALIZAR_PRODUTOS_PELO_CUSTO' 'METRICAS_DO_QUADRO_DE_FUNCIONARIOS'
 'LOCALIZAR_PRODUTOS_PELA_PRODUCAO' 'METRICAS_DE_PRODUCAO_DE_PRODUTOS'
 'LOCALIZAR_FUNCIONARIO' 'DEPARTAMENTO_DO_FUNCIONARIO'
 'MOVIMENTACAO_QUADRO_DE_FUNCIONARIOS' 'TEMPO_DE_CASA_FUNCIONARIO'
 'METRICAS_DE_CUSTO_DE_PRODUTOS']


In [6]:
sentences=data['text']
labels=data['gt']
num_classes=len(data.label.unique())+1
len(sentences),len(labels)

(67, 67)

In [7]:
model_id = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = TFXLMRobertaForSequenceClassification.from_pretrained(model_id,num_labels=num_classes)

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

c:\Users\roger\anaconda3\envs\tensorflow-env-gpu\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\roger\.cache\huggingface\hub\models--xlm-roberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


ResourceExhaustedError: Exception encountered when calling layer "roberta" "                 f"(type TFXLMRobertaMainLayer).

{{function_node __wrapped__AddV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:AddV2]

Call arguments received by layer "roberta" "                 f"(type TFXLMRobertaMainLayer):
  • input_ids=tf.Tensor(shape=(1, 2), dtype=int32)
  • attention_mask=tf.Tensor(shape=(1, 2), dtype=int32)
  • token_type_ids=tf.Tensor(shape=(1, 2), dtype=int32)
  • position_ids=None
  • head_mask=None
  • inputs_embeds=None
  • encoder_hidden_states=None
  • encoder_attention_mask=None
  • past_key_values=None
  • use_cache=None
  • output_attentions=False
  • output_hidden_states=False
  • return_dict=True
  • training=False

In [ ]:
input_ids=[]
attention_masks=[]

for sent in sentences:
    bert_inp=tokenizer.encode_plus(sent,add_special_tokens = True,max_length =64, pad_to_max_length = True,return_attention_mask = True)
    input_ids.append(bert_inp['input_ids'])
    attention_masks.append(bert_inp['attention_mask'])

input_ids=np.asarray(input_ids)
attention_masks=np.array(attention_masks)
labels=np.array(labels)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\Users\roger\anaconda3\envs\tensorflow-env-gpu\lib\site-packages\transformers\tokenization_utils_base.py:2618: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [17]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.debugging.set_log_device_placement(True)

# Create some tensors
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)

Num GPUs Available:  1
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [19]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPU


In [13]:
tf.debugging.set_log_device_placement(True)

In [10]:
train_inp,val_inp,train_label,val_label,train_mask,val_mask=train_test_split(input_ids,labels,attention_masks,test_size=0.2)

NameError: name 'input_ids' is not defined

In [11]:
log_dir='tensorboard_data/tb_bert'
model_save_path='./models/roberta_xlm_model.h5'

callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]

print('\nXLM RoBERTa Model',model.summary())

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5,epsilon=1e-08)

model.compile(loss=loss,optimizer=optimizer,metrics=[metric])

NameError: name 'model' is not defined

In [ ]:
history=model.fit([train_inp,train_mask],train_label,batch_size=1,epochs=4,validation_data=([val_inp,val_mask],val_label),callbacks=callbacks)

Epoch 1/4
53/53 [==============================] - 18s 117ms/step - loss: 2.4240 - accuracy: 0.1698 - val_loss: 2.2038 - val_accuracy: 0.5000
Epoch 2/4
53/53 [==============================] - 5s 86ms/step - loss: 1.9626 - accuracy: 0.5283 - val_loss: 1.6361 - val_accuracy: 0.7143
Epoch 3/4
53/53 [==============================] - 5s 87ms/step - loss: 1.3874 - accuracy: 0.7358 - val_loss: 1.2386 - val_accuracy: 0.6429
Epoch 4/4
53/53 [==============================] - 5s 85ms/step - loss: 0.9624 - accuracy: 0.8868 - val_loss: 0.9674 - val_accuracy: 0.8571


In [14]:
model_id = 'neuralmind/bert-base-portuguese-cased'
log_dir='tensorboard_data/tb_bert'
model_save_path='./models/bertimbau_model.h5'
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5,epsilon=1e-08)

trained_model = TFXLMRobertaForSequenceClassification.from_pretrained(model_id,num_labels=12)
trained_model.compile(loss=loss,optimizer=optimizer, metrics=[metric])
trained_model.load_weights(model_save_path)

preds = trained_model.predict([val_inp,val_mask],batch_size=1)
pred_labels = np.argmax(preds.logits, axis=1)
f1 = f1_score(val_label,pred_labels,average='micro')
print('F1 score',f1)
print('Classification Report')
print(classification_report(val_label,pred_labels,target_names=list(data.label.unique())))

print('Training and saving built model.....')

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier', 'bert/pooler/dense/bias:0', 'bert/pooler/dense/kernel:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


14/14 [==============================] - 5s 25ms/step
F1 score 1.0
Classification Report


ValueError: Number of classes, 6, does not match size of target_names, 11. Try specifying the labels parameter

In [13]:
teste = "Qual total de departamentos"

In [14]:
input_ids=[]
attention_masks=[]

bert_inp=tokenizer.encode_plus(teste,add_special_tokens = True,max_length =64, pad_to_max_length = True,return_attention_mask = True)
input_ids.append(bert_inp['input_ids'])
attention_masks.append(bert_inp['attention_mask'])

input_ids=np.asarray(input_ids)
attention_masks=np.array(attention_masks)

c:\Users\roger\anaconda3\envs\tensorflow-env-gpu\lib\site-packages\transformers\tokenization_utils_base.py:2618: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [16]:

preds = trained_model.predict([input_ids,attention_masks],batch_size=1)


1/1 [==============================] - 0s 27ms/step


In [17]:
preds.logits

array([[-0.6800498 , -0.01390779, -0.07037967, -0.27815893, -0.61175275,
        -0.39857736, -0.39063776, -0.58831465, -0.23552974, -0.56907326,
         3.2777905 , -0.4081495 ]], dtype=float32)

In [18]:
pred_labels = np.argmax(preds.logits, axis=1)

In [19]:
pred_labels

array([10], dtype=int64)

In [20]:
guia = data.label.unique()

In [21]:
guia[pred_labels]

array(['METRICAS_DE_CUSTO_DE_PRODUTOS'], dtype=object)